<a href="https://colab.research.google.com/github/Ariotegar/ariotegar.github.io/blob/main/ToolsDetectionCitraKomp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



*   ARIOTEGAR RAMDHANTO PUTRA (1301213441)
*   DWIKI ARLIMAN (1301213397)





# CELL 1: Mount Gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# CELL 2: Unzip Dataset YOLOv5

In [ ]:
!unzip /content/toolsdataset.zip -d /content/dataset/

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/240_jpg.rf.061b0311131ac0b48ef97cb7aca3e3fa.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/240_jpg.rf.18047e3f58eaab579b2692d6013da9fa.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/240_jpg.rf.3bb9c79241166414a5c8b3292be04a01.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/240_jpg.rf.80d23786dfff2b280ff68b7e675e4721.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/240_jpg.rf.9462076756f7ab237cc7ef2926bbe206.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/240_jpg.rf.a44de1c1a41a940ee160acda86fb1811.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pytorch/train/images/242_jpg.rf.070c183e888bb58534488a5ebe15fd72.jpg  
  inflating: /content/dataset/Tools Detection.v1i.yolov5pyto

# CELL 3: Inisialisasi YOLOv5

In [ ]:
!pip install scikit-image scikit-learn
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
%pip install -qr requirements.txt comet_ml

import torch
import utils
display = utils.notebook_init()



YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)


Setup complete ✅ (12 CPUs, 53.0 GB RAM, 33.2/235.7 GB disk)


# CELL 4: Inisialisasi Pre-Processing dan Feature Extractions

In [ ]:
import os
import cv2
import numpy as np
from google.colab import drive
from pathlib import Path
from tqdm import tqdm
import shutil

class ImagePreprocessor:
    def __init__(self, input_base_dir, output_base_dir):
        self.input_base_dir = input_base_dir
        self.output_base_dir = output_base_dir

    def apply_preprocessing(self, image):
        try:
            # Cek image merupakan BGR
            if len(image.shape) < 3:
                image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

            # Gaussian Blur untuk noise reduction
            denoised = cv2.GaussianBlur(image, (3, 3), 0)

            # Konversi ke LAB color
            lab = cv2.cvtColor(denoised, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)

            # CLAHE kepada Channel L
            clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
            l = clahe.apply(l)

            # Merge back and convert to BGR
            enhanced_lab = cv2.merge([l, a, b])
            enhanced = cv2.cvtColor(enhanced_lab, cv2.COLOR_LAB2BGR)

            return enhanced

        except Exception as e:
            print(f"Error processing image: {str(e)}")
            return image  # Kembalikan image original jika error

    def copy_labels(self, subdir): # Copy paste labels asli ke folder baru menampung image preprocessed
        input_labels_dir = os.path.join(self.input_base_dir, subdir, 'labels')
        output_labels_dir = os.path.join(self.output_base_dir, subdir, 'labels')

        if os.path.exists(input_labels_dir):
            print(f"Copying labels for {subdir}...")
            shutil.copytree(input_labels_dir, output_labels_dir, dirs_exist_ok=True)

    def process_directory(self, subdir): # Preprocess images didalam direktori
        input_dir = os.path.join(self.input_base_dir, subdir, 'images')
        output_dir = os.path.join(self.output_base_dir, subdir, 'images')

        # Membuat direktori
        os.makedirs(output_dir, exist_ok=True)

        # Kopas label
        self.copy_labels(subdir)

        image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

        print(f"Processing {len(image_files)} images in {subdir}...")

        # Pre-Process setiap image
        for img_file in tqdm(image_files):
            try:
                input_path = os.path.join(input_dir, img_file)
                output_path = os.path.join(output_dir, img_file)

                image = cv2.imread(input_path)
                if image is None:
                    print(f"Failed to read {input_path}")
                    continue

                # PreProcessing
                processed = self.apply_preprocessing(image)

                # Menyimpan PreProcessed image
                cv2.imwrite(output_path, processed)

            except Exception as e:
                print(f"Error processing {img_file}: {str(e)}")
                # Copy original file if processing fails
                shutil.copy2(input_path, output_path)

    def process_all(self):
        # Process setiap subdirektori
        for subdir in ['train', 'valid', 'test']:
            try:
                self.process_directory(subdir)
            except Exception as e:
                print(f"Error processing directory {subdir}: {str(e)}")

def extract_features(image_path): # TIDAK DIGUNAKAN
    image = cv2.imread(image_path)
    features = {}

    # 1. Color Features
    # Kalkulasi historigram
    color_hist = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    features['color_hist'] = color_hist.flatten()

    # 2. Texture Features
    # Konversi ke grayscale untuk analisis tekstur
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Kalkulasi GLCM (Gray-Level Co-occurrence Matrix)
    glcm = cv2.GaussianBlur(gray, (3, 3), 0)  # Simplification for example
    features['texture'] = np.array(glcm).flatten()

    # 3. Shape Features
    # Kalkulasi fitur edge menggunakan Canny
    edges = cv2.Canny(gray, 100, 200)
    features['edges'] = np.sum(edges) / (edges.shape[0] * edges.shape[1])

    return features

# CELL 5: Preprocess dataset

In [ ]:
preprocessor = ImagePreprocessor(
    input_base_dir='/content/dataset/Tools Detection.v1i.yolov5pytorch',
    output_base_dir='/content/preprocesseddataset'
)

preprocessor.process_all()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Copying labels for train...
Processing 2850 images in train...


100%|██████████| 2850/2850 [00:41<00:00, 68.77it/s]


Copying labels for valid...
Processing 236 images in valid...


100%|██████████| 236/236 [00:03<00:00, 69.59it/s]


Copying labels for test...
Processing 5 images in test...


100%|██████████| 5/5 [00:00<00:00, 68.20it/s]


# CELL 6: Training

In [ ]:
# Training dataset menggunakan epoch 50
!python train.py --img 640 --batch 64 --epochs 50 --data /content/dataset/data.yaml --weights yolov5s6.pt --cache

2024-12-13 10:44:57.057105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-13 10:44:57.077959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-13 10:44:57.084305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=yolov5s6.pt, cfg=, data=/content/dataset/data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=64, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=data/hyps, resume_evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single

# CELL 7: Validasi

In [ ]:
!python val.py --data '/content/dataset/Tools Detection.v1i.yolov5pytorch/data.yaml' --weights /content/drive/MyDrive/AIStuff/CitraKompTrainingResult/weights/best.pt

val: data=/content/dataset/Tools Detection.v1i.yolov5pytorch/data.yaml, weights=['/content/drive/MyDrive/AIStuff/CitraKompTrainingResult/weights/best.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 206 layers, 12327460 parameters, 0 gradients, 16.2 GFLOPs
val: Scanning /content/preprocesseddataset/valid/labels... 236 images, 0 backgrounds, 0 corrupt: 100% 236/236 [00:00<00:00, 3412.81it/s]
val: New cache created: /content/preprocesseddataset/valid/labels.cache
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 8/8 [00:06<00:00,  1.29it/s]
                   all        236        298

# CELL 7: Test Deteksi

In [ ]:
!python detect.py --source /content/preprocesseddataset/test/images --weights /content/drive/MyDrive/AIStuff/CitraKompTrainingResult/weights/best.pt

detect: weights=['/content/drive/MyDrive/AIStuff/CitraKompTrainingResult/weights/best.pt'], source=/content/preprocesseddataset/test/images, data=data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-389-ge62a31b6 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 206 layers, 12327460 parameters, 0 gradients, 16.2 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/5 /content/preprocesseddataset/test/images/105_jpg.rf.96673c915105517fb527188ae88f44a6.jpg: 640x640 2 Handss, 1 Plier, 8.2ms
image 2/5 /content/preprocesseddataset/test/images/108_jpg.rf.9f9c5e225d9390

# CELL 8: Feature Extraction (Tidak digunakan karena menggunakan YOLOv5)

In [ ]:
features = extract_features('/content/preprocesseddataset/test/images/105_jpg.rf.96673c915105517fb527188ae88f44a6.jpg')